In [ ]:
from os import environ
from twitter import OAuth

oauth = OAuth(environ['ACCESS_TOKEN'],
              environ['ACCESS_SECRET'],
              environ['CONSUMER_KEY'],
              environ['CONSUMER_SECRET'])

In [ ]:
from twitter import TwitterStream

los_angeles_bbox = "-118.55, 33.97, -118.44, 34.05"
twitterator = (TwitterStream(auth=oauth)
               .statuses
               .filter(locations=los_angeles_bbox))

In [ ]:
tw = next(twitterator)

In [ ]:
tw.keys()

In [ ]:
import pymongo

mongo_server = pymongo.MongoClient('this_mongo')

In [ ]:
mongo_server.twitter.tweets.count()

In [ ]:
result = (mongo_server
          .twitter
          .tweets
          .insert_one(tw))

In [ ]:
mongo_server.twitter.tweets.count()

### Encode Tweets as Document Vectors

In [ ]:
import spacy

nlp = spacy.load('en')

In [ ]:
cursor = mongo_server.twitter.tweets.find()
stored_tweet = cursor.next()

In [ ]:
stored_tweet['text']

In [ ]:
doc = nlp(stored_tweet['text'])

In [ ]:
doc.vector.shape

In [ ]:
mongo_server.twitter.tweets.update_one(
    {'_id': stored_tweet['_id']}, 
    {'$set': {'document_vector': doc.vector.tostring()}})

In [ ]:
stored_tweet = cursor.next()
stored_tweet['text']

In [ ]:
doc = nlp(stored_tweet['text'])

In [ ]:
mongo_server.twitter.tweets.update_one(
    {'_id': stored_tweet['_id']}, 
    {'$set': {'document_vector': doc.vector.tostring()}}, 
    upsert=False)